In [1]:
import pandas as pd
import numpy as np

risk_df = pd.read_csv("stock_risk_summary.csv")
risk_df.head()


,Stock,Avg_Daily_Return,Avg_20D_Volatility
0,AAPL,0.000798,0.018192
1,ABBV,0.001054,0.016847
2,ACN,-0.001177,0.017161
3,ADBE,-0.001122,0.019817
4,ADI,0.000495,0.021966


In [2]:
# avoid division issues
risk_df = risk_df.replace([np.inf, -np.inf], np.nan).dropna()

# risk-adjusted score
risk_df['Risk_Adjusted_Score'] = (
    risk_df['Avg_Daily_Return'] / risk_df['Avg_20D_Volatility']
)


In [3]:
risk_df['Risk_Adjusted_Score'] = risk_df['Risk_Adjusted_Score'].clip(lower=0)


In [4]:
total_score = risk_df['Risk_Adjusted_Score'].sum()

risk_df['Portfolio_Weight'] = (
    risk_df['Risk_Adjusted_Score'] / total_score
)


In [5]:
risk_df['Portfolio_Weight'].sum()


1.0

In [6]:
weights_df = risk_df[[
    'Stock',
    'Avg_Daily_Return',
    'Avg_20D_Volatility',
    'Risk_Adjusted_Score',
    'Portfolio_Weight'
]].sort_values(
    'Portfolio_Weight',
    ascending=False
)

weights_df.head(10)


,Stock,Avg_Daily_Return,Avg_20D_Volatility,Risk_Adjusted_Score,Portfolio_Weight
58,MO,0.001500,0.012126,0.123735,0.033842
64,NFLX,0.002327,0.019482,0.119420,0.032662
35,GE,0.002165,0.018701,0.115751,0.031658
48,JPM,0.001729,0.015420,0.112140,0.030670
40,GS,0.002157,0.019365,0.111376,0.030462
18,C,0.002095,0.018897,0.110873,0.030324
76,SCHW,0.001707,0.015939,0.107118,0.029297
60,MS,0.001819,0.018838,0.096555,0.026408
25,CSCO,0.001280,0.013281,0.096352,0.026352
38,GOOG,0.001863,0.019797,0.094081,0.025731


In [8]:
# Portfolio weight ko percentage mein convert
weights_df["Portfolio_Weight_Percent"] = weights_df["Portfolio_Weight"] * 100

# sirf clean columns rakhen
final_weights_df = weights_df[[
    'Stock',
    'Avg_Daily_Return',
    'Avg_20D_Volatility',
    'Risk_Adjusted_Score',
    'Portfolio_Weight_Percent'
]]

# round for readability
final_weights_df = final_weights_df.round({
    'Avg_Daily_Return': 6,
    'Avg_20D_Volatility': 6,
    'Risk_Adjusted_Score': 6,
    'Portfolio_Weight_Percent': 2
})

# CSV save
final_weights_df.to_csv(
    "portfolio_weights_percentage.csv",
    index=False
)

# quick check
final_weights_df.head(10)


,Stock,Avg_Daily_Return,Avg_20D_Volatility,Risk_Adjusted_Score,Portfolio_Weight_Percent
58,MO,0.001500,0.012126,0.123735,3.38
64,NFLX,0.002327,0.019482,0.119420,3.27
35,GE,0.002165,0.018701,0.115751,3.17
48,JPM,0.001729,0.015420,0.112140,3.07
40,GS,0.002157,0.019365,0.111376,3.05
18,C,0.002095,0.018897,0.110873,3.03
76,SCHW,0.001707,0.015939,0.107118,2.93
60,MS,0.001819,0.018838,0.096555,2.64
25,CSCO,0.001280,0.013281,0.096352,2.64
38,GOOG,0.001863,0.019797,0.094081,2.57


In [9]:
## Correlation matrix

In [10]:
data = pd.read_csv("clean_sp100_data.csv", parse_dates=["Date"])

# sirf Daily Return columns nikaalo
return_cols = [c for c in data.columns if c.endswith("_Daily Return")]

# stock-wise returns dataframe
returns_df = data[return_cols].copy()

# column names clean karo: AAPL_Daily Return -> AAPL
returns_df.columns = [c.replace("_Daily Return", "") for c in returns_df.columns]

# correlation matrix
correlation_matrix = returns_df.corr()

# CSV save
correlation_matrix.to_csv(
    "stock_return_correlation_matrix.csv"
)

# quick check
correlation_matrix.iloc[:5, :5]

,AAPL,MSFT,AMZN,GOOGL,GOOG
AAPL,1.000000,0.514982,0.582052,0.482131,0.486479
MSFT,0.514982,1.000000,0.666873,0.500970,0.504413
AMZN,0.582052,0.666873,1.000000,0.597489,0.594776
GOOGL,0.482131,0.500970,0.597489,1.000000,0.997957
GOOG,0.486479,0.504413,0.594776,0.997957,1.000000
